## 一个简单的装饰器
## 标准库中的装饰器


In [43]:
# 头文件
import time
import functools
import html
import collections
import numbers

In [15]:
def clock(func):
    def clocked(*args):
        t_0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t_0
        name = func.__name__
        arg_str = ", ".join(repr(arg) for arg in args)
        print('[%0.8fs]%s(%s)->%r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [16]:
@clock
def snooze(seconds):
    time.sleep(seconds)


@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

snooze(4)
factorial(100)

print(factorial.__name__)



[3.99962140s]snooze(4)->None
[0.00000140s]factorial(1)->1
[0.00005710s]factorial(2)->2
[0.00008570s]factorial(3)->6
[0.00011390s]factorial(4)->24
[0.00013990s]factorial(5)->120
[0.00016530s]factorial(6)->720
[0.00018880s]factorial(7)->5040
[0.00021180s]factorial(8)->40320
[0.00024070s]factorial(9)->362880
[0.00026680s]factorial(10)->3628800
[0.00029190s]factorial(11)->39916800
[0.00031460s]factorial(12)->479001600
[0.00033680s]factorial(13)->6227020800
[0.00036630s]factorial(14)->87178291200
[0.00039190s]factorial(15)->1307674368000
[0.00041750s]factorial(16)->20922789888000
[0.00044760s]factorial(17)->355687428096000
[0.00047070s]factorial(18)->6402373705728000
[0.00049970s]factorial(19)->121645100408832000
[0.00052240s]factorial(20)->2432902008176640000
[0.00054550s]factorial(21)->51090942171709440000
[0.00057130s]factorial(22)->1124000727777607680000
[0.00059470s]factorial(23)->25852016738884976640000
[0.00061950s]factorial(24)->620448401733239439360000
[0.00064510s]factorial(25)->1

### 装饰器的典型行为：
- 把被装饰的函数替换成新函数
- 二者接受相同的参数
- 赶回被装饰的函数本该返回的值，同时还会做一些额外的操作

In [17]:
#import functools

# Demo_7-17
def clock_2(func):

    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t_0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t_0
        name = func.__name__
        args_list = []
        if args:
            args_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            args_list.append(', '.join(pairs))
        arg_str = ', '.join(args_list)
        print('[%0.8fs]%s(%s)->%r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [22]:
@clock_2
def snooze(seconds):
    time.sleep(seconds)


@clock_2
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

snooze(4)
factorial(10)

print(factorial.__name__)

[3.99936560s]snooze(4)->None
[0.00000140s]factorial(1)->1
[0.00010270s]factorial(2)->2
[0.00101920s]factorial(3)->6
[0.00110930s]factorial(4)->24
[0.00116010s]factorial(5)->120
[0.00120600s]factorial(6)->720
[0.00125180s]factorial(7)->5040
[0.00129230s]factorial(8)->40320
[0.00133750s]factorial(9)->362880
[0.00138480s]factorial(10)->3628800
factorial


In [24]:
@clock_2
def fibonacci(n):
    return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000040s]fibonacci(0)->0
[0.00000030s]fibonacci(1)->1
[0.00004250s]fibonacci(2)->1
[0.00000020s]fibonacci(1)->1
[0.00000070s]fibonacci(0)->0
[0.00000020s]fibonacci(1)->1
[0.00009630s]fibonacci(2)->1
[0.00012130s]fibonacci(3)->2
[0.00018040s]fibonacci(4)->3
[0.00000020s]fibonacci(1)->1
[0.00000020s]fibonacci(0)->0
[0.00000030s]fibonacci(1)->1
[0.00001790s]fibonacci(2)->1
[0.00003840s]fibonacci(3)->2
[0.00000020s]fibonacci(0)->0
[0.00000020s]fibonacci(1)->1
[0.00001480s]fibonacci(2)->1
[0.00000020s]fibonacci(1)->1
[0.00000020s]fibonacci(0)->0
[0.00000020s]fibonacci(1)->1
[0.00001740s]fibonacci(2)->1
[0.00003420s]fibonacci(3)->2
[0.00006410s]fibonacci(4)->3
[0.00011880s]fibonacci(5)->5
[0.00031650s]fibonacci(6)->8
8


In [25]:
# fibonacci 中多次调用相同内容，浪费时间。
# 使用lru_cache记录计算结果
@functools.lru_cache()
@clock_2
def fibonacci_2(n):
    return n if n < 2 else fibonacci_2(n-2) + fibonacci_2(n-1)

print(fibonacci_2(6))

[0.00000040s]fibonacci_2(0)->0
[0.00000030s]fibonacci_2(1)->1
[0.00005410s]fibonacci_2(2)->1
[0.00000050s]fibonacci_2(3)->2
[0.00007250s]fibonacci_2(4)->3
[0.00000040s]fibonacci_2(5)->5
[0.00009150s]fibonacci_2(6)->8
8


### lru_cache（）
- 参数
    - maxsize：制定存储多少个调用结果，设为2的幂数
    - typed：如果为true，把不同类型得到的结果分开保存
- 注意：
    - 被装饰的函数的参数，都必须是可散列的。

In [28]:
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [44]:
print(htmlize({1, 2, 3}))

print(htmlize(abs))

print(htmlize(42))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<pre>42</pre>


In [52]:
@functools.singledispatch
def htmlize_2(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize_2.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize_2.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:o})</pre>'.format(n)

@htmlize_2.register(tuple)
@htmlize_2.register(collections.abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join((htmlize_2(item) for item in seq))
    return '<ul>\n<li>' + inner + '</li>\n</ul>'



In [56]:
print(htmlize_2('hello world \n'))

print(htmlize_2(100))

print(htmlize_2((1,2,3,4)))

<p>hello world <br>
</p>
<pre>100 (0x144)</pre>
<ul>
<li><pre>1 (0x1)</pre></li>
<li><pre>2 (0x2)</pre></li>
<li><pre>3 (0x3)</pre></li>
<li><pre>4 (0x4)</pre></li>
</ul>


### 叠放装饰器

- 先执行离函数近的装饰器

### 装饰器参数
- 装饰器工厂函数

In [58]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' % (active, func.__name__))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate


@register(False)
def func_1():
    print('running func_1')

@register(True)
def func_2():
    print('running func_2')

running register(active=False)->decorate(func_1)
running register(active=True)->decorate(func_2)


In [61]:
func_1()
print('func_1, registry:', registry)
func_2()
print('func_2, registry:', registry)



running func_1
func_1, registry: {<function func_2 at 0x0000027083BB2D30>}
running func_2
func_2, registry: {<function func_2 at 0x0000027083BB2D30>}


In [62]:
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args})->{result}'
def clock_3(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*args, **kwargs):
            t_0 = time.perf_counter()
            result = func(*args, **kwargs)
            elapsed = time.perf_counter() - t_0
            name = func.__name__
            args_list = []
            if args:
                args_list.append(', '.join(repr(arg) for arg in args))
            if kwargs:
                pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
                args_list.append(', '.join(pairs))
            arg_str = ', '.join(args_list)
            # print(fmt.format(elapsed=elapsed, name=name, args=arg_str, result=result))
            print(fmt.format(**locals()))
            return result
        return clocked
    return decorate


@clock_3()
def snooze(seconds):
    time.sleep(seconds)

In [64]:
for i in range(5):
    snooze(i/10)

[0.00000820s] snooze((0.0,))->None
[0.10026900s] snooze((0.1,))->None
[0.19997480s] snooze((0.2,))->None
[0.30032070s] snooze((0.3,))->None
[0.40077160s] snooze((0.4,))->None


## 第七章 End

